<a href="https://colab.research.google.com/github/Pentuhov/PR.AI-1/blob/main/%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D1%8B%D0%B9_%D1%81%D0%BB%D0%BE%D0%B9_(Dense)_%D0%94%D0%97_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задание

Создайте систему компьютерного зрения, которая будет определять тип геометрической фигуры. Используя подготовленную базу и шаблон ноутбука проведите серию экспериментов по перебору гиперпараметров нейронной сети, распознающей три категории изображений (треугольник, круг, квадрат).

1. Поменяйте количество нейронов в сети, используя следующие значения:

- один слой 10 нейронов
- один слой 100 нейронов
- один слой 5000 нейронов.

2. Поменяйте активационную функцию в скрытых слоях с `relu` на `linear`.
3. Поменяйте размеры batch_size:
- 10
- 100
- 1000

4. Выведите на экран получившиеся точности.

Всего должно получиться 18 комбинаций указанных параметров.

Создайте сравнительную таблицу по результатам проведенных тестов.

In [ ]:
# Подключение класса для создания нейронной сети прямого распространения
from tensorflow.keras.models import Sequential
# Подключение класса для создания полносвязного слоя
from tensorflow.keras.layers import Dense
# Подключение оптимизатора
from tensorflow.keras.optimizers import Adam
# Подключение утилит для to_categorical
from tensorflow.keras import utils
# Подключение библиотеки для загрузки изображений
from tensorflow.keras.preprocessing import image
# Подключение библиотеки для работы с массивами
import numpy as np
# Подключение библиотек для отрисовки изображений
import matplotlib.pyplot as plt
# Подключение модуля для работы с файлами
import os
# Преобразование изображения в одномерный массив
from tensorflow.keras.layers import Flatten
# Вывод изображения в ноутбуке, а не в консоли или файле
%matplotlib inline

In [ ]:
# Загрузка датасета из облака
import gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l3/hw_light.zip', None, quiet=True)

'hw_light.zip'

In [ ]:
# Распаковываем архив hw_light.zip в папку hw_light
import zipfile
with zipfile.ZipFile('hw_light.zip', 'r') as zip_ref:
    zip_ref.extractall(r'C:\Users\Terion\Занятие 1\Домашняя работа')

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Путь к директории с базой
base_dir = r'C:\Users\Terion\Занятие 1\Домашняя работа\hw_light'
# Создание пустого списка для загрузки изображений обучающей выборки
x_train = []
# Создание списка для меток классов
y_train = []
# Задание высоты и ширины загружаемых изображений
img_height = 20
img_width = 20
# Перебор папок в директории базы
for patch in os.listdir(base_dir):
    # Перебор файлов в папках
    for img in os.listdir(base_dir + '/' + patch):
        # Добавление в список изображений текущей картинки
        x_train.append(image.img_to_array(image.load_img(base_dir + '/' + patch + '/' + img,
                                                    target_size=(img_height, img_width),
                                                    color_mode='grayscale')))
        # Добавление в массив меток, соответствующих классам
        if patch == '0':
            y_train.append(0)
        elif patch == '3':
            y_train.append(1)
        else:
            y_train.append(2)

# Преобразование в numpy-массив загруженных изображений и меток классов
x_train = np.array(x_train)
y_train = np.array(y_train)
# Вывод размерностей
print('Размер массива x_train', x_train.shape)
print('Размер массива y_train', y_train.shape)

Размер массива x_train (302, 20, 20, 1)
Размер массива y_train (302,)


In [ ]:
# Разделение данных на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# Функция для создания и обучения модели с заданными гиперпараметрами
def create_and_train_model(neurons, activation_func, batch_size):
    model = Sequential()
    model.add(Flatten(input_shape=(img_height, img_width, 1)))
    model.add(Dense(neurons, activation=activation_func))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2, verbose=2)

    # Оценка точности на тестовой выборке
    y_pred = np.argmax(model.predict(x_test), axis=1)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [ ]:
# Эксперименты с разными гиперпараметрами
neuron_counts = [10, 100, 5000]
activation_functions = ['relu', 'linear']  # Сначала relu, затем linear
batch_sizes = [10, 100, 1000]

results = []

for neurons in neuron_counts:
    for activation_func in activation_functions:
        for batch_size in batch_sizes:
            print(f"\nЭксперимент с {neurons} нейронами, активационной функцией {activation_func} и batch_size={batch_size}:")
            accuracy = create_and_train_model(neurons, activation_func, batch_size)
            results.append((neurons, activation_func, batch_size, accuracy))
            print(f"Точность: {accuracy}")


Эксперимент с 10 нейронами, активационной функцией relu и batch_size=10:
Epoch 1/10


C:\Users\Terion\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 - 2s - 82ms/step - accuracy: 0.3802 - loss: 63.3004 - val_accuracy: 0.3469 - val_loss: 29.3748
Epoch 2/10
20/20 - 0s - 11ms/step - accuracy: 0.4167 - loss: 3.3570 - val_accuracy: 0.3673 - val_loss: 1.0970
Epoch 3/10
20/20 - 0s - 13ms/step - accuracy: 0.3542 - loss: 1.0819 - val_accuracy: 0.3878 - val_loss: 1.0980
Epoch 4/10
20/20 - 0s - 12ms/step - accuracy: 0.3490 - loss: 1.0839 - val_accuracy: 0.3265 - val_loss: 1.0980
Epoch 5/10
20/20 - 0s - 18ms/step - accuracy: 0.3594 - loss: 1.0788 - val_accuracy: 0.3265 - val_loss: 1.0980
Epoch 6/10
20/20 - 0s - 11ms/step - accuracy: 0.3594 - loss: 1.0772 - val_accuracy: 0.3265 - val_loss: 1.0981
Epoch 7/10
20/20 - 0s - 12ms/step - accuracy: 0.3646 - loss: 1.0798 - val_accuracy: 0.3265 - val_loss: 1.0982
Epoch 8/10
20/20 - 0s - 11ms/step - accuracy: 0.3646 - loss: 1.0778 - val_accuracy: 0.3265 - val_loss: 1.0981
Epoch 9/10
20/20 - 0s - 11ms/step - accuracy: 0.3646 - loss: 1.0769 - val_accuracy: 0.3265 - val_loss: 1.0981
Epoch 10/10
20/20 -

C:\Users\Terion\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 1s - 624ms/step - accuracy: 0.3229 - loss: 256.8353 - val_accuracy: 0.4286 - val_loss: 132.8005
Epoch 2/10
2/2 - 0s - 67ms/step - accuracy: 0.3177 - loss: 132.6236 - val_accuracy: 0.3878 - val_loss: 68.5403
Epoch 3/10
2/2 - 0s - 69ms/step - accuracy: 0.2865 - loss: 84.6207 - val_accuracy: 0.3878 - val_loss: 68.9011
Epoch 4/10
2/2 - 0s - 74ms/step - accuracy: 0.2969 - loss: 76.1472 - val_accuracy: 0.3673 - val_loss: 56.9754
Epoch 5/10
2/2 - 0s - 71ms/step - accuracy: 0.3229 - loss: 52.7232 - val_accuracy: 0.4898 - val_loss: 30.0553
Epoch 6/10
2/2 - 0s - 68ms/step - accuracy: 0.3438 - loss: 31.8762 - val_accuracy: 0.5102 - val_loss: 26.0418
Epoch 7/10
2/2 - 0s - 73ms/step - accuracy: 0.4531 - loss: 32.4855 - val_accuracy: 0.5510 - val_loss: 28.1186
Epoch 8/10
2/2 - 0s - 63ms/step - accuracy: 0.4688 - loss: 32.6726 - val_accuracy: 0.5306 - val_loss: 25.2454
Epoch 9/10
2/2 - 0s - 71ms/step - accuracy: 0.4375 - loss: 26.6163 - val_accuracy: 0.4898 - val_loss: 22.6288
Epoch 10/10
2/2 -

...

20/20 - 1s - 53ms/step - accuracy: 0.4167 - loss: 1613.4240 - val_accuracy: 0.6939 - val_loss: 558.1386
Epoch 2/10
20/20 - 0s - 18ms/step - accuracy: 0.7031 - loss: 381.4403 - val_accuracy: 0.6735 - val_loss: 288.1711
Epoch 3/10
20/20 - 0s - 18ms/step - accuracy: 0.7604 - loss: 148.4454 - val_accuracy: 0.5714 - val_loss: 226.4767
Epoch 4/10
20/20 - 0s - 18ms/step - accuracy: 0.7292 - loss: 76.9409 - val_accuracy: 0.6531 - val_loss: 149.6150
Epoch 5/10
20/20 - 1s - 37ms/step - accuracy: 0.7292 - loss: 68.8464 - val_accuracy: 0.6735 - val_loss: 161.3628
Epoch 6/10
20/20 - 0s - 23ms/step - accuracy: 0.7969 - loss: 60.2880 - val_accuracy: 0.6531 - val_loss: 206.5271
Epoch 7/10
20/20 - 0s - 22ms/step - accuracy: 0.7969 - loss: 36.1324 - val_accuracy: 0.6735 - val_loss: 136.0905
Epoch 8/10
20/20 - 0s - 21ms/step - accuracy: 0.7969 - loss: 41.5801 - val_accuracy: 0.5918 - val_loss: 167.2534
Epoch 9/10
20/20 - 0s - 21ms/step - accuracy: 0.6979 - loss: 84.1712 - val_accuracy: 0.5510 - val_loss:

C:\Users\Terion\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 - 1s - 639ms/step - accuracy: 0.2969 - loss: 1317.0326 - val_accuracy: 0.3878 - val_loss: 4535.9756
Epoch 2/10
2/2 - 0s - 75ms/step - accuracy: 0.3229 - loss: 4432.3208 - val_accuracy: 0.3878 - val_loss: 1867.1421
Epoch 3/10
2/2 - 0s - 82ms/step - accuracy: 0.3281 - loss: 1997.1488 - val_accuracy: 0.4286 - val_loss: 2104.3411
Epoch 4/10
2/2 - 0s - 97ms/step - accuracy: 0.4844 - loss: 1551.4242 - val_accuracy: 0.3061 - val_loss: 2235.1675
Epoch 5/10
2/2 - 0s - 85ms/step - accuracy: 0.3385 - loss: 1904.2479 - val_accuracy: 0.3469 - val_loss: 1120.5015
Epoch 6/10
2/2 - 0s - 80ms/step - accuracy: 0.5677 - loss: 682.8723 - val_accuracy: 0.5918 - val_loss: 493.3088
Epoch 7/10
2/2 - 0s - 89ms/step - accuracy: 0.5885 - loss: 486.9934 - val_accuracy: 0.5510 - val_loss: 893.3822
Epoch 8/10
2/2 - 0s - 75ms/step - accuracy: 0.6406 - loss: 747.2644 - val_accuracy: 0.3469 - val_loss: 1224.5696
Epoch 9/10
2/2 - 0s - 91ms/step - accuracy: 0.5156 - loss: 862.9195 - val_accuracy: 0.5102 - val_loss: 

C:\Users\Terion\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 - 1s - 1s/step - accuracy: 0.3698 - loss: 102.8219 - val_accuracy: 0.3265 - val_loss: 4615.9746
Epoch 2/10
1/1 - 0s - 96ms/step - accuracy: 0.3438 - loss: 4041.8025 - val_accuracy: 0.4082 - val_loss: 3019.8904
Epoch 3/10
1/1 - 0s - 98ms/step - accuracy: 0.5104 - loss: 2524.8523 - val_accuracy: 0.4694 - val_loss: 1829.1649
Epoch 4/10
1/1 - 0s - 97ms/step - accuracy: 0.5573 - loss: 1514.6498 - val_accuracy: 0.4490 - val_loss: 518.8458
Epoch 5/10
1/1 - 0s - 96ms/step - accuracy: 0.5365 - loss: 359.3551 - val_accuracy: 0.3878 - val_loss: 1703.9146
Epoch 6/10
1/1 - 0s - 95ms/step - accuracy: 0.3229 - loss: 1642.2080 - val_accuracy: 0.3878 - val_loss: 2138.7466
Epoch 7/10
1/1 - 0s - 97ms/step - accuracy: 0.3229 - loss: 2068.4014 - val_accuracy: 0.4286 - val_loss: 1635.3817
Epoch 8/10
1/1 - 0s - 94ms/step - accuracy: 0.4271 - loss: 1531.9614 - val_accuracy: 0.4898 - val_loss: 837.2173
Epoch 9/10
1/1 - 0s - 97ms/step - accuracy: 0.4844 - loss: 794.2274 - val_accuracy: 0.3061 - val_loss: 10

In [ ]:
# Преобразование результатов в DataFrame для удобной обработки
df_results = pd.DataFrame(results, columns=['Neurons', 'Activation Function', 'Batch Size', 'Accuracy'])

In [ ]:
# Создание пустого DataFrame для сравнительной таблицы
df_comparison = pd.DataFrame(columns=['Neurons', 'Batch Size', 'Accuracy with ReLU', 'Accuracy with Linear'])

In [ ]:
# Заполнение сравнительной таблицы
for neurons in neuron_counts:
    for batch_size in batch_sizes:
        relu_accuracy = df_results[(df_results['Neurons'] == neurons) & (df_results['Activation Function'] == 'relu') & (df_results['Batch Size'] == batch_size)]['Accuracy'].values[0]
        linear_accuracy = df_results[(df_results['Neurons'] == neurons) & (df_results['Activation Function'] == 'linear') & (df_results['Batch Size'] == batch_size)]['Accuracy'].values[0]
        df_comparison = pd.concat([df_comparison, pd.DataFrame({'Neurons': [neurons], 'Batch Size': [batch_size], 'Accuracy with ReLU': [relu_accuracy], 'Accuracy with Linear': [linear_accuracy]})], ignore_index=True)

print("\nСравнительная таблица результатов:")
print(df_comparison)


Сравнительная таблица результатов:
  Neurons Batch Size  Accuracy with ReLU  Accuracy with Linear
0      10         10            0.278689              0.704918
1      10        100            0.442623              0.426230
2      10       1000            0.508197              0.377049
3     100         10            0.754098              0.590164
4     100        100            0.622951              0.491803
5     100       1000            0.508197              0.426230
6    5000         10            0.737705              0.475410
7    5000        100            0.508197              0.573770
8    5000       1000            0.606557              0.508197
